In [178]:
import re
import smtplib
from datetime import datetime
import requests
from bs4 import BeautifulSoup as bs
import os
import re

In [179]:
# Get unique values
def unique(xlist):
    unique_list = []
    
    for x in xlist:
        if x not in unique_list:
            unique_list.append(x)
    
    return unique_list

In [180]:
def search(list, keyword):
    for i in range(len(list)):
        if list[i] == keyword:
            return True
    return False

In [181]:
query_url = "https://www.xe.gr/property/results?transaction_name=rent&item_type=re_residence&maximum_price=960&minimum_size=57&geo_lat_from=38.0999672763109&geo_lng_from=24.061083822481493&geo_lat_to=37.792875685371634&geo_lng_to=23.530055641375128&sorting=update_desc"
crafted_urls = []
uniq_urls = []
start_page = 0  # Should be 1 at minimum
max_page_num = 3  # upper limit for the web page number to scrapp
links = []

for i in range(start_page, max_page_num):
    crafted_urls.append( query_url + "&page=" + str(i+1) )
    soup = bs(requests.get(crafted_urls[i]).content, 'lxml')
    
    for a_tag in soup.select('a[href*="enoikiaseis-katoikion"]'):
        links.append(a_tag['href'])
        
    links = links[:-15]


In [182]:
len(links)

204

In [183]:
uniq_urls = unique(links)

In [184]:
len(uniq_urls)

102

In [185]:
blacklist_areas = [
    "nosokomeio-paidwn", 
    "athhna-plateia-amerikhs",
    "vyrwnas",
    "zwgrafoy-kentro",
    "nosokomeio-paidwn",
    "athhna-koyntoyriwtika",
    "athhna-kynosargoys",
    "galatsi",
    "porto-rafth",
    "pagkrati",
    "ippokratoys",
    "ilion",
    "liosia",
    "keratsini"
    ]

In [186]:
blacklist_areas = unique(blacklist_areas)
blacklist_areas

['nosokomeio-paidwn',
 'athhna-plateia-amerikhs',
 'vyrwnas',
 'zwgrafoy-kentro',
 'athhna-koyntoyriwtika',
 'athhna-kynosargoys',
 'galatsi',
 'porto-rafth',
 'pagkrati',
 'ippokratoys',
 'ilion',
 'liosia',
 'keratsini']

In [187]:
def filter_blacklisted_areas(urls, blacklist_areas):
    # Filters based on blacklisted areas
        
    for url in urls:
        for area in blacklist_areas:
            if area in url:
                urls.remove(url)
                break # if blacklisted break the loop
    return urls

In [188]:
cleaned_urls= filter_blacklisted_areas(uniq_urls, blacklist_areas)

In [189]:
print(len(cleaned_urls))

95


In [355]:
class Home:
    def __init__(self, property_url):
        self.property_url = property_url
        self.property_id = -1
        self.bedroom = -1
        self.bathroom = -1
        self.floor = -1
        self.house_reconstruction = "NA"
        self.building_space = "NA"
        self.compass = "NA"
        self.title = "NA"
        self.price = -1
        self.door = False
        self.canopy = False
        self.heating = "NA"
        self.is_estate = False
        self.description = "NA"
        self.has_oil = False
        self.rejected = True
        
    def set_property_id(self):
        self.property_id = self.property_url.split('/')[6]
        
    def house_print_info(self):
        print(f'Ηλεκτρονική Διεύθυνση: {self.property_url}')
        print(f'Αναγνωριστικό: {self.property_id}')
        print(f'Υπνοδωμάτια: {self.bedroom}')
        print(f'Μπάνια: {self.bathroom}')
        print(f'Όροφος: {self.floor}')
        print(f'Έτος ανακαίνισης: {self.house_reconstruction}')
        print(f'Κατάσταση: {self.building_space}')
        print(f'Προσανατολισμός: {self.compass}')
        print(f'Τίτλος: {self.title}')
        print(f'Τιμή: {self.price}')
        print(f'Πόρτα ασφαλείας: {self.door}')
        print(f'Τέντες: {self.canopy}')
        print(f'Θέρμανση: {self.heating}')
        print(f'Μεσίτης: {self.is_estate}')
        print(f'Περιγραφή: {self.description}')
        print(f'Πετρέλαιο: {self.has_oil}')


    def set_house_info(self, soup_html, soup_lxml):
        
        if search(self.text, "Υπνοδωμάτια:"):
            self.bedroom = int( self.text[self.text.index("Υπνοδωμάτια:") + 2].text )

        if search(self.text, "Μπάνια:"):
            self.bathroom = int( self.text[self.text.index("Μπάνια:") + 2].text )

        floor_data = self.text[self.text.index("Όροφος:") + 2].text
        if type(floor_data) == list and len(floor_data) > 1:
            floor_data = floor_data[0]
        if ((floor_data == 'Ισόγειο') or (floor_data == 'Υπερυψωμένο, Ισόγειο')):
            self.floor = 0
        elif floor_data == 'Ημιώροφος':
            self.floor = 1/2
        else:
            digits = re.findall(r'\d+', floor_data)
            if type(digits) == list and len(digits) > 0:
                digits = digits[0]
            self.floor = int(str(digits))

        if search(self.text, "Έτος ανακαίνισης:"):
            self.house_reconstruction = int(self.text[self.text.index("Έτος ανακαίνισης:") + 2].text)

        if search(self.text, "Κατάσταση:"):
            self.building_space = self.text[self.text.index("Κατάσταση:") + 2].text

        if search(self.text, "Προσανατολισμός:"):
            self.compass = self.text[self.text.index("Προσανατολισμός:") + 2].text

        self.title = soup_lxml.title.string
        self.price = int (re.findall(r'\d+', soup_lxml.find(class_='price').h2.text)[0])
        self.door = search(self.text, "Πόρτα ασφαλείας")
        self.canopy = search(self.text, "Τέντες")

        if search(self.text, "Θέρμανση:"):
            self.heating = self.text[self.text.index("Θέρμανση:") + 2].text

        if self.title.find('Estate') != -1:
            self.is_estate = True
        elif self.title.find('Brokers') != -1:
            self.is_estate = True
        elif self.title.find('Ιδιώτης') != -1:
            self.is_estate = False
        elif self.title.find('Property') != -1:
            self.is_estate = True
        elif self.title.find('Group') != -1:
             self.is_estate = True
        elif (len(soup_html.find_all("h2", {"class": "title"})) != 0):
            title = soup_html.find_all("h2", {"class": "title"})[0].text
            print(f'title: {title} len: {len(title)}\n')
            
            if (title == 'broker'):
                self.is_estate = True

        self.description = soup_lxml.p.string

        if self.description:
            if ((self.description.find('πετρέλαιο') != -1) or (self.description.find('πετρελαίου') != -1)):
                self.has_oil = True

    def set_text(self, soup_html):
        parent = soup_html.find("body").find("ul")  # finding parent <ul> tag
        self.text = list(parent.descendants)  # finding all <li> tags
        
    def validate(self):
        if not self.has_oil and not self.is_estate:
            self.rejected = False

In [356]:
def make_homes(urls):
    Homes = []

    for i in range(0, len(urls)):
        Homes.append(Home(urls[i]))
        Homes[i].set_property_id()
    
    return Homes

In [357]:
Homes = make_homes(cleaned_urls)
len(Homes)

95

In [358]:
print(Homes[3].property_id)
print(Homes[3].property_url)

785766383
https://www.xe.gr/property/d/enoikiaseis-katoikion/785766383/petroypolh-khpoypolh-600-95?first_in_widget_ad_id=785765828&rank_in_widget=4&widget_name=search+results+list


In [359]:
def get_soup_objects(url):
    
    # creating requests object
    html = requests.get(url).content
        
    # creating soup objects
    soup_html = bs(html, 'html.parser')
    soup_lxml = bs(html, 'lxml')
    
    return [soup_html, soup_lxml]

In [362]:
def parse_and_filter_homes():
    selected_houses = []
    selected = False
    
    for h in Homes:
        print(f'\nProcessing house with id: {h.property_id} and url: {h.property_url}')
        
        [soup_html, soup_lxml] = get_soup_objects(h.property_url)
        
        # find tags and set text
        h.set_text(soup_html) 
        
        h.set_house_info(soup_html, soup_lxml)
        
        h.validate()
        
        
        if not h.rejected:
            h.house_print_info()
            selected_houses.append(h)
            print(f'This house matches out criteria!')
            print(f'Addind it to our database...')
            print(f'Informing you via email...')
        
    
    return selected_houses

In [363]:
selected_houses = parse_and_filter_homes()


Processing house with id: 785765828 and url: https://www.xe.gr/property/d/enoikiaseis-katoikion/785765828/athhna-lamprinh-400-60?first_in_widget_ad_id=785765828&rank_in_widget=1&widget_name=search+results+list


AttributeError: 'NoneType' object has no attribute 'descendants'

In [306]:
url='https://www.xe.gr/property/d/enoikiaseis-katoikion/785796801/petroypolh-kentro-600-65?first_in_widget_ad_id=785765828&rank_in_widget=15&widget_name=search+results+list'
html = requests.get(url).content
soup_html = bs(html, 'html.parser')
soup_lxml = bs(html, 'lxml')
parent = soup_html.find("body").find("ul")
text = list(parent.descendants)


In [307]:
bedroom = int( text[text.index("Υπνοδωμάτια:") + 2].text )
bedroom

2

In [308]:
bathroom="NA"
if search(text, "Μπάνια:"):
    bathroom = int( text[text.index("Μπάνια:") + 2].text )
bathroom

1

In [309]:
text[text.index("Όροφος:") + 2]

<span>1ος</span>

In [310]:
text[text.index("Όροφος:") + 2].text

'1ος'

In [311]:
l = ['Υπερυψωμένο, Ισόγειο', '']
type(l)

list

In [312]:
floor_data = text[text.index("Όροφος:") + 2].text
print(type(floor_data))
print(floor_data)

if type(floor_data) == list and len(floor_data) > 1:
    floor_data = floor_data[0]
re.findall(r'\d+', floor_data)

<class 'str'>
1ος


['1']

In [313]:
floor_data = text[text.index("Όροφος:") + 2].text
if type(floor_data) == list and len(floor_data) > 1:
    floor_data = floor_data[0]
    
if ((floor_data == 'Ισόγειο') or (floor_data == 'Υπερυψωμένο, Ισόγειο')):
    floor = 0
elif floor_data == 'Ημιώροφος':
    floor = 1/2
else:
    digits = re.findall(r'\d+', floor_data)
    if type(digits) == list and len(digits) > 1:
        digits = digits[0]
        floor = int(str(digits))
floor

NameError: name 'floor' is not defined

In [314]:
house_reconstruction="NA"
if search(text, "Έτος ανακαίνισης:"):
    house_reconstruction = int(text[text.index("Έτος ανακαίνισης:") + 2].text)
house_reconstruction

2022

In [315]:
building_space="NA"
if search(text, "Κατάσταση:"):
    building_space = text[text.index("Κατάσταση:") + 2].text
building_space

'Ανακαινισμένο'

In [316]:
compass="NA"
if search(text, "Προσανατολισμός:"):
    compass = text[text.index("Προσανατολισμός:") + 2].text
compass

'NA'

In [317]:
door = search(text, "Πόρτα ασφαλείας")
door

True

In [318]:
canopy = search(text, "Τέντες")
canopy

True

In [319]:
heating="NA"
if search(text, "Θέρμανση:"):
    heating = text[text.index("Θέρμανση:") + 2].text
heating

'Αυτόνομη θέρμανση με ατομική εγκατάσταση'

In [320]:
title = soup_lxml.title.string
title

'\n  Διαμέρισμα προς ενοικίαση 65 τ.μ. Πετρούπολη (Κέντρο) (Ιδιώτης) | xe.gr\n'

In [321]:
is_estate = False
if title.find('Estate') != -1:
    is_estate = True
print(is_estate)

False


In [322]:
is_private = False
if (len(soup_html.find_all("h2", {"class": "title"})) != 0):
    if (soup_html.find_all("h2", {"class": "title"})[0].text == 'Αγγελία ιδιώτη'):
        is_private = True
is_private

False

In [164]:
# written like this in order to face a known bug
if (is_private):
    print('Ιδιώτης')
elif (is_estate):
    print('Μεσίτης')
else:
    print('Ιδιώτης')

Μεσίτης


In [165]:
description = soup_lxml.p.string
description

'\n                    1 κουζίνα/ες, κουφώματα: Αλουμινίου, Ασανσέρ, Ηλ. Συσκευές, Διπλά Τζάμια, Metro, Φωτεινό, Επι Λεωφόρου, Ανακαινισμένο, Ενοικιάζετε πλήρως επιπλωμένο, κοντά σε: Μεταξουργείο. Plasis Σύνταγμα, Τηλ. Επικοινωνίας: +30 2103224225, email: athens@plasis.com.gr, website: www.plasis.com.gr\n                  '

In [166]:
price = int (re.findall(r'\d+', soup_lxml.find(class_='price').h2.text)[0])
price

650

In [167]:
has_oil = False
if ((description.find('πετρέλαιο') != -1) or (description.find('πετρελαίου') != -1)):
    has_oil = True
has_oil

False